In [ ]:
import math
import os
import re
from os import path

import spacy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

DIRETORIO_PDF = 'BauruPDF/'
DIRETORIO_TXT = 'BauruTXT/'
DIRETORIO_DOC = 'BauruDOC/'
DIRETORIO_PAR = 'BauruPAR/'
DIRETORIO_JSON = 'BauruJSON/'

nlp_exclude_list = ['parser',
                    'ner',
                    'entity_linker',
                    'entity_ruler',
                    'textcat',
                    'textcat_multilabel',
                    'lemmatizer',
                    'trainable_lemmatizer',
                    'morphologizer',
                    'attribute_ruler',
                    'senter',
                    'sentencizer',
                    'tok2vec',
                    'transformer']
nlp = spacy.load('pt_core_news_lg', exclude = nlp_exclude_list)

In [ ]:
def similaridade_cosseno(a, b):
    """
    Calcula a similaridade de cosseno entre dois dicts.
    Supõe-se que as chaves são iguais, compondo "vetores" a e b em um mesmo espaço.
    Como se trata de sacos de palavras, os vetores terão apenas valores inteiros positivos,
    logo a similaridade não poderão assumir similaridades negativas.
    
    :param a dict: "vetor" a
    :param b dict: "vetor" b
    
    :returns: valor no intervalo [0, 1] similaridade cosseno entre os "vetores" a e b.
    """
    
    assert type(a) == dict
    assert type(b) == dict
    assert a.keys() == b.keys()
    
    produto_escalar  = 0
    soma_a = 0
    soma_b = 0
    
    for k in a:
        produto_escalar  += a[k] * b[k]
        soma_a           += math.pow(a[k], 2)
        soma_b           += math.pow(b[k], 2)

    magnitude = math.sqrt(soma_a) * math.sqrt(soma_b)
    
    if abs(magnitude) < 1e-3:
        similaridade = 0
    else:
        similaridade = produto_escalar / magnitude
    
    return similaridade

def atualiza_chaves(d, ks):
    """
    Atualiza as chaves do dict d para conter também as chaves ks.
    
    :param d dict: dict a ser atualizado
    :param ks dict_keys: chaves a serem adicionadas em d
    
    :returns: dict d
    """
    
    for k in ks:
        if k not in d:
            d[k] = 0

    return d

def token_valido(token):
    """
    Verifica se um token é valido para ser incluído no saco de palavras
    
    :param token spacy.tokens.Token: token a ser verificado
    
    :returns: bool se é válido ou não
    """
    
    assert type(token) == spacy.tokens.Token
    
    return not (token.is_stop
               or token.is_space
               or token.is_punct
               or token.is_currency
               or token.is_quote
               or token.is_bracket
               or token.like_num
               or token.is_oov)

def monta_saco_palavras(doc, saco = None):
    """
    Monta/atualiza o saco de palavras de um doc com os tokens válidos da função token_valido.
    
    :param doc spacy.tokens.Doc: Doc a ser utilizado no saco de palavras.
    :keyword saco dict: saco de palavras a ser atualizado, ou criado, caso não especificado
    
    :returns: saco de palavras criado, ou atualizado
    """
    
    if saco == None:
        saco = dict()
    
    for token in doc:
        if token_valido(token):
            chave = token.norm_
            if chave not in saco:
                saco[chave] = 0
            saco[chave] += 1
        else:
            pass
    return saco

def calcula_similaridade_doc(doc, saco_alvo_template):
    """
    Calcula a similaridade do saco de um Doc com outro saco previamente gerado.
    
    :param doc spacy.tokens.Doc: Doc a ser analisado.
    :param saco_alvo_template dict: saco a ser comparado.
    
    :returns: similaridade dos sacos gerados
    """
    
    saco_alvo = saco_alvo_template.copy()
    saco_doc = {}
    
    atualiza_chaves(saco_doc, saco_alvo.keys())
    monta_saco_palavras(doc, saco_doc)
    atualiza_chaves(saco_alvo, saco_doc.keys())
    
    similaridade = similaridade_cosseno(saco_alvo, saco_doc)
    
    return similaridade


extrator_numero_ano = re.compile('((?P<numero>\d+(\.\d{3})?)/(?P<ano>\d{2,4}))')
extrator_nome_numero_ano = re.compile('((?P<nome>\D+)\s+(?P<numero>(\d+)(\.\d{3})?)/(?P<ano>\d{2,4}))')

In [9]:

vocabulario = {}
vocabulario_pub = {}
modalidades = {}
tipos = {}

for filename in tqdm(os.listdir(DIRETORIO_JSON)):
    with open(path.join(DIRETORIO_JSON, filename), 'r') as file:
        licitacao = json.load(file)

    if licitacao['titulo']['modalidade'] not in modalidades:
        modalidades[licitacao['titulo']['modalidade']] = 0
    modalidades[licitacao['titulo']['modalidade']] += 1
    
    tipos_key = licitacao['titulo']['modalidade'] + licitacao['tipo']
    if tipos_key not in tipos:
        tipos[tipos_key] = 0
    tipos[tipos_key] += 1
        
    if 'publicacoes' in licitacao:
        for p in licitacao['publicacoes']:
            p_chave = p['titulo']
            if p_chave not in vocabulario:
                vocabulario[p_chave] = {}
            
            doc = nlp(p['conteudo'])
            for token in doc:
                if token_valido(token):
                    chave = token.norm_
                    
                    if chave not in vocabulario[p_chave]:
                        vocabulario[p_chave][chave] = 0
                    if chave not in vocabulario_pub:
                        vocabulario_pub[chave] = 0
                        
                    vocabulario[p_chave][chave] += 1
                    vocabulario_pub[chave] += 1
                    


  6%|▋         | 223/3522 [00:04<01:01, 53.59it/s]


KeyboardInterrupt: 

In [13]:
#sample = pd.DataFrame(os.listdir(DIRETORIO_PAR), columns=['filename']).sample(2)

kw_matcher = spacy.matcher.PhraseMatcher(nlp.vocab, attr = 'NORM')
kw_matcher.add('EDITAL', [nlp('Edital')])
kw_matcher.add('PROCESSO', [nlp('Processo')])
kw_matcher.add('MODALIDADE', [nlp('Modalidade')])
kw_matcher.add('OBJETO', [nlp('Objeto')])

edital_id = spacy.strings.hash_string('EDITAL')
processo_id = spacy.strings.hash_string('PROCESSO')
modalidade_id = spacy.strings.hash_string('MODALIDADE')
objeto_id = spacy.strings.hash_string('OBJETO')

for filename in sample['filename']:
    print(filename)
    with open(path.join(DIRETORIO_PAR, filename), 'r') as file:
        paragrafos = json.load(file)

    for paragrafo in paragrafos:
        doc = nlp(paragrafo)
        #doc = spacy.tokens.Doc(nlp.vocab).from_json(paragrafo)
        matches = kw_matcher(doc)
        
        if len(matches) > 0:
            tem_edital = False
            tem_modalidade = False
            tem_processo = False
            
            for match_id, start, end in matches:
                if (match_id == edital_id
                    or match_id == processo_id
                    or match_id == modalidade_id):
                    
                    seguintes = doc[end:end + 10]
                    
                    re_match = None
                    
                    if match_id == modalidade_id:
                        re_match = extrator_nome_numero_ano.search(seguintes.text)
                    else:
                        re_match = extrator_numero_ano.search(seguintes.text)

                    if re_match != None:
                        print(f'{nlp.vocab.strings[match_id]}', end = '')
                        if match_id == edital_id:
                            tem_edital = True
                        elif match_id == processo_id:
                            tem_processo = True
                        elif match_id == modalidade_id:
                            tem_modalidade = True
                            print(f'{re_match.group("nome")}')

                        print(f'\t{re_match.group("numero")}\t{re_match.group("ano")}')
                        
            if tem_processo and not (tem_edital or tem_modalidade):
                print(doc)
            print('################################')


do_20170812_2863.pdf.xhtml.json
PROCESSO	44.538	2017
(artigo 26 da Lei Federal 8.666, de 21 de junho de 1.993)
Ratifico a Dispensa de Licitação para Locação do imóvel situado na Rua Araújo Leite, nº 17-47, nesta cidade de Bauru, 
Estado de São Paulo, de propriedade de CARDEC BATISTA FONTANA RUFINO, neste ato representado pela 
SDT MORAES – SERVIÇOS DE APOIO ADMINISTRATIVO E COBRANÇAS LTDA, destinado a abrigar a 
sede da Secretaria Municipal de Economia e Finanças, no valor total de R$ 468.000,00 (quatrocentos e sessenta e oito 
mil reais), com fundamento no artigo 24, inciso X, da Lei Federal nº 8.666, de 21 de junho de 1.993 e suas alterações 
posteriores, de acordo com justificativa de fls.01 e tendo em vista os elementos que instruem o Processo Administrativo 
nº 44.538/2017.
Bauru, 11 de agosto de 2017.

################################
PROCESSO	16.322	17
EDITAL	162	17
MODALIDADE: PREGÃO PRESENCIAL 
Nº
	022	17
################################
PROCESSO	16.175	17
EDITAL	161	17
MODALI

MODALIDADE: Convite nº
	020	17
################################
EDITAL	22	08
################################
EDITAL	263	17
PROCESSO	36.492	17
MODALIDADE: Pregão Eletrônico nº
	178	17
################################
EDITAL	271	17
PROCESSO	9.673	17
MODALIDADE: Convite nº
	018	17
################################
EDITAL	251	2017
PROCESSO	10.609	2017
MODALIDADE: 
Convite nº
	016	2017
################################
EDITAL	165	2017
PROCESSO	22.803	2017
MODALIDADE: Convite nº
	005	2017
################################
EDITAL	170	17
PROCESSO	21.220	2017
MODALIDADE: 
Concorrência Pública nº
	002	2017
################################
EDITAL	170	17
################################
EDITAL	484	16
PROCESSO	3.464	2010
MODALIDADE: 
Concorrência Pública nº
	010	16
EDITAL	484	16
################################
EDITAL	255	2017
PROCESSO	36.731	2017
MODALIDADE: Convite nº
	017	2017
EDITAL	255	2017
EDITAL	255	17
################################
EDITAL	150	17
PROCESSO	8.345	17
MODALIDADE: Pregão Eletrôni

EDITAL	03	2017
PROCESSO	185	2017
MODALIDADE: Pregão Presencial nº
	03	2017
################################
PROCESSO	001	2014
Atos da Presidência
ESTÁGIO PROBATÓRIO: Foram aprovados os servidores abaixo indicados na avaliação de Estágio 
Probatório referente ao Processo RH-001/2014, neste mês de Agosto/2017, nos cargos efetivos: 

################################
################################
################################
do_20220315_3526.pdf.xhtml.json
PROCESSO	31.917	2022
DISPENSA: A partir 02/03/2022, portaria nº 340/2022, dispensa a pedido, o(a) servidor(a) FABIANA 
VIEIRA SOLFA, RG nº 34xxxxx39, matrícula nº 33812, da função de confiança de Diretor da Divisão  
do Programa Saúde da Família -PSF, da SECRETARIA MUNICIPAL DA SAÚDE, conforme Processo 
nº 31.917/2022.

################################
PROCESSO	31.145	2022
DISPENSA: A partir 01/03/2022, portaria nº 341/2022, dispensa a pedido, o(a) servidor(a) MARIAH 
REINATO FERRAO, RG nº 41xxxxx08, matrícula nº 35260, da função 

In [29]:
"""
############################################################################
DETECTOR DE NÚMERO DE PROCESSOS
Compara os dados das publicações com os dados estruturados da página web
############################################################################
"""

sample = pd.DataFrame(os.listdir(DIRETORIO_JSON), columns = ['filename'])#.sample(100)

kw_matcher = spacy.matcher.PhraseMatcher(nlp.vocab, attr = 'NORM')
kw_matcher.add('EDITAL', [nlp('Edital')])
kw_matcher.add('PROCESSO', [nlp('Processo')])
kw_matcher.add('MODALIDADE', [nlp('Modalidade')])
kw_matcher.add('OBJETO', [nlp('Objeto')])

edital_id = spacy.strings.hash_string('EDITAL')
processo_id = spacy.strings.hash_string('PROCESSO')
modalidade_id = spacy.strings.hash_string('MODALIDADE')
objeto_id = spacy.strings.hash_string('OBJETO')

for filename in sample['filename']:
    with open(path.join(DIRETORIO_JSON, filename), 'r') as file:
        licitacao = json.load(file)
    
#     if licitacao['titulo']['modalidade'] != 'Dispensa':
#         continue
    
    if 'publicacoes' not in licitacao:
        continue
        
    processo = licitacao['processo']
    
    for publicacao in licitacao['publicacoes']:
        doc = nlp(publicacao['conteudo'])
        matches = kw_matcher(doc)    
        
        if len(matches) > 0:
            for match_id, start, end in matches:
                if match_id == processo_id:
                    teste_numero = doc[start:end + 7]
                    re_match = extrator_numero_ano.search(teste_numero.text)

                    if re_match != None:
                        num_json = int(processo['numero'].replace('.', ''))
                        num_pub = int(re_match.group('numero').replace('.', ''))
                        
                        if num_json != num_pub:
                            print(filename)
                            print(licitacao['titulo']['modalidade'], licitacao['titulo']['numero'], licitacao['titulo']['ano'])
    
                            print('NUMEROS DIFERENTES:')
                            print(f'{processo["numero"]}/{processo["ano"]} vs {re_match.group("numero")}/{re_match.group("ano")}')
                            print(f'{num_json} vs {num_pub}')
                            print(teste_numero)
                            print()
        


3918.json
Dispensa 104 2016
NUMEROS DIFERENTES:
53.391/2016 vs 52.391/2016
53391 vs 52391
Processo: 52.391/2016 – Modalidade: Dispensa de

6177.json
Pregão Eletrônico 183 2020
NUMEROS DIFERENTES:
112.757/2019 vs 112.575/19
112757 vs 112575
Processo n.º 112.575/19 - Modalidade: Pregão Eletrônico

5781.json
Pregão Eletrônico 410 2019
NUMEROS DIFERENTES:
103.098/2019 vs 365/368
103098 vs 365
processo fls. 365/368 dos autos, INDEFERIU

5941.json
Dispensa 15 2019
NUMEROS DIFERENTES:
104.084/2019 vs 18/06
104084 vs 18
processo. Bauru, 18/06/2021 – Davison de

6146.json
Pregão Eletrônico 135 2020
NUMEROS DIFERENTES:
138.343/2019 vs 16/10
138343 vs 16
PROCESSO	R$ 38.450,00
Bauru, 16/10/2020

6112.json
Pregão Eletrônico 159 2020
NUMEROS DIFERENTES:
43.141/2020 vs 43.145/2020
43141 vs 43145
Processo nº 43.145/2020 - CONTRATANTE: MUNICÍPIO DE

6112.json
Pregão Eletrônico 159 2020
NUMEROS DIFERENTES:
43.141/2020 vs 43.145/2020
43141 vs 43145
processo 43.145/2020, do Decreto nº 13.093/2016,

5972.j

6261.json
Pregão Eletrônico 296 2020
NUMEROS DIFERENTES:
82.040/2020 vs 287/296
82040 vs 287
processo administrativo em epígrafe fls. 287/296,

6261.json
Pregão Eletrônico 296 2020
NUMEROS DIFERENTES:
82.040/2020 vs 287/296
82040 vs 287
processo administrativo em epígrafe fls. 287/296,

5542.json
Dispensa 49 2019
NUMEROS DIFERENTES:
89.217/2019 vs 114.860/2019
89217 vs 114860
Processo 114.860/2019 – Modalidade: Dispensa de Licitação

5102.json
Pregão Eletrônico 375 2018
NUMEROS DIFERENTES:
57.187/2018 vs 374/377
57187 vs 374
processo fls. 374/377, NEGA provimento ao

4143.json
Pregão Eletrônico 74 2017
NUMEROS DIFERENTES:
56.944/2016 vs 65.877/16
56944 vs 65877
PROCESSO n.º 65.877/16;
LEIA-SE: NOTIFICAÇÃO

5682.json
Pregão Eletrônico 339 2019
NUMEROS DIFERENTES:
12.132/2018 vs 579/581
12132 vs 579
processo fls. 579/581 dos autos, INDEFERIU



In [38]:
modalidades

{'Pregão Eletrônico': 2705,
 'Concorrência Pública': 103,
 'Inexigibilidade': 118,
 'Pregão Presencial': 155,
 'Dispensa': 344,
 'Convite': 83,
 'Tomada de Preços': 11,
 'Leilão': 2,
 'Concurso': 1}

In [39]:
tipos

{'Pregão EletrônicoNota de Empenho': 343,
 'Pregão EletrônicoRegistro de Preço': 1597,
 'Concorrência PúblicaContrato': 101,
 'Pregão EletrônicoContrato': 765,
 'InexigibilidadeNota de Empenho': 44,
 'Pregão PresencialRegistro de Preço': 70,
 'DispensaNota de Empenho': 147,
 'DispensaProcesso Emergencial': 42,
 'Pregão PresencialContrato': 76,
 'DispensaChamamento Público': 94,
 'ConviteContrato': 71,
 'DispensaProcesso Direto': 32,
 'InexigibilidadeContrato': 60,
 'InexigibilidadeChamamento Público': 10,
 'DispensaContrato': 29,
 'ConviteNota de Empenho': 12,
 'Concorrência PúblicaRegistro de Preço': 2,
 'Tomada de PreçosContrato': 11,
 'InexigibilidadeProcesso Direto': 4,
 'Pregão PresencialNota de Empenho': 9,
 'LeilãoContrato': 1,
 'ConcursoContrato': 1,
 'LeilãoNota de Empenho': 1}